# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,245.59,97,64,3.05,RU,1739127333
1,1,bonthe,7.5264,-12.5050,300.37,73,6,4.45,SL,1739127334
2,2,stanley,54.8680,-1.6985,277.33,89,75,3.60,GB,1739127336
3,3,keflavik,64.0049,-22.5624,279.39,93,100,11.83,IS,1739127337
4,4,port-aux-francais,-49.3500,70.2167,280.98,94,94,14.95,TF,1739127338


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',
    size='Humidity',
    scale=1,
    color='City',
    frame_width=800,
    frame_height=500,
    geo=True,
    tiles='OSM'  # OpenStreetMap tiles to include state outlines
).opts(
    title='City Humidity'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
%%capture --no-display
# Narrow down cities that fit criteria and drop any results with null values
# Narrow down cities that fit the ideal weather criteria
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] >= 20) & (city_data_df["Humidity"] <= 25) & (city_data_df["Cloudiness"] < 60) & (city_data_df["Wind Speed"] < 65)]

# Drop any rows with null values
cleaned_df = ideal_weather_df.dropna()

# Configure the map plot
map_plot = cleaned_df.hvplot.points(
    'Lng', 'Lat',
    size='ideal_weather',
    scale=1,
    color='City',
    frame_width=800,
    frame_height=500,
    geo=True,
    tiles='OSM'  # OpenStreetMap tiles to include state outlines
).opts(
    title='Ideal Weather Cities'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
print(hotel_df.head())

                City Country      Lat       Lng  Humidity Hotel Name
0              tiksi      RU  71.6872  128.8694        97           
1             bonthe      SL   7.5264  -12.5050        73           
2            stanley      GB  54.8680   -1.6985        89           
3           keflavik      IS  64.0049  -22.5624        93           
4  port-aux-francais      TF -49.3500   70.2167        94           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 100000  # Increase the search radius to 20,000 meters (20 km)
params = {
    "categories": "accommodation.hotel",
    "limit": 3,  # Try to get up to 3 hotels to check availability
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search


tiksi - nearest hotel: Арктика
bonthe - nearest hotel: No hotel found
stanley - nearest hotel: Hotel 52
keflavik - nearest hotel: Núpan Deluxe
port-aux-francais - nearest hotel: Keravel
bestobe - nearest hotel: No hotel found
albany - nearest hotel: Tru
sitka - nearest hotel: Aspen Suites Hotel
prince rupert - nearest hotel: Crest Hotel
eirunepe - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: Lava Tree Tropic Inn
waitangi - nearest hotel: Hotel Chathams
adamstown - nearest hotel: No hotel found
khandyga - nearest hotel: Апельсин
villemandeur - nearest hotel: Hôtel inn
whitehorse - nearest hotel: Town & Mountain Hotel
saldanha - nearest hotel: Hoedjesbaai
chibougamau - nearest hotel: No hotel found
newport - nearest hotel: Queens Hotel
blackmans bay - nearest hotel: Villa Howden
bardai - nearest hotel: No hotel found
papatowai - nearest hotel: Catlins Woodstock Lodge & Camp
saipan - nearest hotel: Chalan Kanoa Beach Hotel
fiambala - nearest hotel: Casona del Pino

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tiksi,RU,71.6872,128.8694,97,Арктика
1,bonthe,SL,7.5264,-12.5050,73,No hotel found
2,stanley,GB,54.8680,-1.6985,89,Hotel 52
3,keflavik,IS,64.0049,-22.5624,93,Núpan Deluxe
4,port-aux-francais,TF,-49.3500,70.2167,94,Keravel
...,...,...,...,...,...,...
575,bahraigh,IN,27.5833,81.6000,40,Golden Palms Nanpara
576,gisborne,NZ,-38.6533,178.0042,84,Portside Hotel
577,alice springs,AU,-23.7000,133.8833,36,Aurora Alice Springs
578,kidal,ML,18.4411,1.4078,17,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_plot=hotel_df.hvplot.points(
    "Lng", 
    "Lat",
    hover_cols=["Hotel Name", "Country"],
    color="City",
    tiles="OSM"
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)